# Get Started
- The script is used to ingest document to RAG, which is a semi-manual process.
- Before running the script, install dependencies with the following command in the current directory of this file `pip3 install -r requirements.txt`
- RAG ingestion has two parts: 1) document preparation; 2) write document to the corresponding DB environment

# High level steps to prepare documents
1. Create csv file with the following schema `organization_id,title,content,url`. `organization_id` should be tied to the user profile. `title` is the title of the document. `content` is the actual content of the document. `url` is the source of the document, optional field.
2. Manually prepare the entries. One row corresponds to one document.
3. Refer to sample file `willy.csv`

# High level steps to write documents to DB
1. Read document from csv file. The csv schema should follow the one mentioned above. 
2. Generate embeddings
3. Generate chunks
4. Connect to Qdrant DB
5. Write the document and embeddings to DB
6. Validate with test search query
7. Validate with Qdrant DB web portal

In [1]:
import openai
import os
import pandas as pd
import numpy as np
import json
import tiktoken
import ast
import math
# Get openAI api key by reading local .env file
from dotenv import load_dotenv, find_dotenv
from qdrant_client import QdrantClient
_ = load_dotenv(find_dotenv()) 
openai.api_key  = os.environ['OPENAI_API_KEY']

/Users/tingzhenming/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load your CSV file into a pandas DataFrame
df = pd.read_csv('willy.csv')
df.head()

,organization_id,title,content,url
0,willyCompany,PRD - User management v1,Product Requirements Document\n\nProblem & Opp...,http://localhost:3000/docs/clvps7amb000a3gwwmz...


In [ ]:
# Experimental: ingest database schema to RAG
# TODO: automate the process of loading code and producing eventual content
file_path = '../server/db/schema.prisma'
with open(file_path, 'r') as file:
    file_content = file.read()

# Note: it is important to 1) provide high level information including code format; 2) use ``` to wrap the code content
content = 'this is the database schema definition of Willy in the format of prisma file. ```' + file_content + '```'
row = ['willyCompany','schema.prisma', content, 'NA']
df = pd.DataFrame([row], columns=['organization_id','title','content','url'])
df.head()

In [3]:
# Helper functions to help us create the embeddings

# Helper func: calculate number of tokens
def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    try:
        num_tokens = len(encoding.encode(string, disallowed_special=()))
    except Exception as e:
        print(f"Error encode {string}: {e}")
        return 0
    return num_tokens

# Helper function: calculate length of essay
def get_essay_length(essay):
    word_list = essay.split()
    num_words = len(word_list)
    return num_words

# Helper function: calculate cost of embedding num_tokens
# Assumes we're using the text-embedding-ada-002 model
# See https://openai.com/pricing
def get_embedding_cost(num_tokens):
    return num_tokens/1000*0.0001

# Helper function: calculate total cost of embedding all content in the dataframe
def get_total_embeddings_cost():
    total_tokens = 0
    for i in range(len(df.index)):
        text = df['content'][i]
        token_len = num_tokens_from_string(text)
        total_tokens = total_tokens + token_len
    total_cost = get_embedding_cost(total_tokens)
    return total_cost

# quick check on total token amount for price estimation
total_cost = get_total_embeddings_cost()
print("estimated price to embed this content = $" + str(total_cost))

estimated price to embed this content = $6.97e-05


In [4]:
# Create new list with small content chunks to not hit max token limits
# Note: the maximum number of tokens for a single request is 8191
# https://openai.com/docs/api-reference/requests

# list for chunked content and embeddings
new_list = []
ideal_token_size = 4096
# Split up the text into token sizes of around 512 tokens
for i in range(len(df.index)):
    text = df['content'][i]
    token_len = num_tokens_from_string(text)
    if token_len == 0:
        print("skipping " + str(text))
        continue
    if token_len <= ideal_token_size:
        new_list.append([df['organization_id'][i],df['title'][i], df['content'][i], df['url'][i], token_len])
    else:
        # add content to the new list in chunks
        start = 0
        # 1 token ~ 3/4 of a word
        ideal_size = int(ideal_token_size // (4/3))
        end = ideal_size
        #split text by spaces into words
        words = text.split()

        #remove empty spaces
        words = [x for x in words if x != ' ']

        total_words = len(words)
        
        #calculate iterations
        chunks = total_words // ideal_size
        if total_words % ideal_size != 0:
            chunks += 1
        
        new_content = []
        for j in range(chunks):
            if end > total_words:
                end = total_words
            new_content = words[start:end]
            new_content_string = ' '.join(new_content)
            new_content_token_len = num_tokens_from_string(new_content_string)
            if new_content_token_len > 0:
                new_list.append([df['organization_id'][i],df['title'][i], new_content_string, df['url'][i], new_content_token_len])
            start += ideal_size
            end += ideal_size

print(new_list)

[['willyCompany', 'PRD - User management v1', 'Product Requirements Document\n\nProblem & Opportunity\n\nThe current user management system in Project Willy is limited to email/password login and lacks support for OAuth-based authentication methods like Google OAuth. Additionally, the existing system does not provide a comprehensive way for users to join different teams through invitations, nor does it allow team admins to manage team members effectively. The new enterprise-grade user management system for Omniflow users will address these gaps by introducing Google OAuth signup and login, team invitation mechanisms, and enhanced admin capabilities for managing team members and user account statuses.\n\nTarget Audience & User Persona\n\nThe target audience for the new user management system includes:\n\n\n\n\n\nCompany Admins: Responsible for managing user accounts, including suspending or deactivating accounts.\n\n\n\nTeam Admins: Responsible for managing team members, including addin

In [5]:
# Helper function: get embeddings for a text
from openai import OpenAI
client = OpenAI()
def get_embeddings(text):
   return client.embeddings.create(input = [text], model="text-embedding-ada-002").data[0].embedding

# Create embeddings for each piece of content
for i in range(len(new_list)):
   text = new_list[i][2]
   embedding = get_embeddings(text)
   new_list[i].append(embedding)

# Create a new dataframe from the list
df_new = pd.DataFrame(new_list, columns=['organization_id','title', 'content', 'url', 'tokens', 'embeddings'])
df_new.head()

,organization_id,title,content,url,tokens,embeddings
0,willyCompany,PRD - User management v1,Product Requirements Document\n\nProblem & Opp...,http://localhost:3000/docs/clvps7amb000a3gwwmz...,697,"[0.007016368210315704, -0.025088831782341003, ..."


In [6]:
# connect to qdrant database and create a new collection
# 1536 is the size of OpenAI text-embedding-3-small embedding model
from qdrant_client import QdrantClient, models
from qdrant_client.http import models as rest
import uuid

# Get db connection by reading local .env file
_ = load_dotenv(find_dotenv()) 
api_key  = os.environ['QDRANT_API_KEY']
connection_url  = os.environ['QDRANT_DATABASE_URL'] 
qdrant_client = QdrantClient(
    url=connection_url,
    api_key=api_key,
)
print(os.environ['QDRANT_DATABASE_URL'])
collection_name = "text-collection"
# qdrant_client.create_collection(
#     collection_name=collection_name,
#     vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
# )


https://f26a0111-a0c4-428c-84be-1b2c98869ba0.us-east4-0.gcp.cloud.qdrant.io


In [7]:
qdrant_client.upsert(
    collection_name=collection_name,
    points=[
        rest.PointStruct(
            id=str(uuid.uuid4()),
            vector=v['embeddings'],
            payload={
                "organization_id": v['organization_id'],
                "title": v['title'],
                "url": v['url'],
                "content": v['content'],
            },
        )
        for k, v in df_new.iterrows()
    ],
)

UpdateResult(operation_id=9, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# !Use with Caution! Copy data from dev to prod db
prod_qdrant_url = '<PROD_URL>'
prod_qdrant_key = '<PROD_KEY>'
prod_qdrant_client = QdrantClient(
            url = prod_qdrant_url,
            api_key = prod_qdrant_key)
# prod_qdrant_client.create_collection(
#     collection_name=collection_name,
#     vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
# )
print(prod_qdrant_url)

In [ ]:
# !Use with Caution! Copy data from dev to prod db
# qdrant_client.migrate(prod_qdrant_client,[collection_name],batch_size = 100,recreate_on_collision=True)

In [9]:
# Helper function: Get top 3 most similar documents from the database
# remember to update the organization_id to match the organization_id in the database
def get_top3_similar_docs(query_embedding, qdrant_client):
    embedding_array = np.array(query_embedding)
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=embedding_array,
        query_filter=models.Filter(
            must=[models.FieldCondition(key="organization_id", match=models.MatchValue(value="willyCompany"))]
        ),
        with_payload=True,
        limit=3,
    )
    return [v.payload['content'] for v in search_result]

# Helper function: get text completion from OpenAI API
# Note we're using the latest gpt-3.5-turbo-0613 model
from openai import OpenAI
client = OpenAI()

def get_completion_from_messages(messages, model="gpt-3.5-turbo-0613", temperature=0, max_tokens=1000):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response

# Helper function: get embeddings for a text
def get_embeddings(text):
    return client.embeddings.create(input = [text.replace("\n"," ")], model="text-embedding-ada-002").data[0].embedding

# Function to process input with retrieval of most similar documents from the database
def process_input_with_retrieval(user_input):
    delimiter = "```"

    #Step 1: Get documents related to the user input from database
    related_docs = get_top3_similar_docs(get_embeddings(user_input), qdrant_client)
    print(related_docs)
    # Step 2: Get completion from OpenAI API
    # Set system message to help set appropriate tone and context for model
    system_message = f"""
    You are a friendly chatbot. \
    You can answer questions about products, its features and its use cases. \
    You respond in a concise, technically credible tone. \
    """

    # Prepare messages to pass to model
    # We use a delimiter to help the model understand the where the user_input starts and ends
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{delimiter}{user_input}{delimiter}"},
        {"role": "assistant", "content": f"Relevant information: \n {related_docs[0]}"}   
    ]

    final_response = get_completion_from_messages(messages).choices[0].message.content
    return final_response


input = "What is needed to build a enterprise grade user management system for Omniflow?"
response = process_input_with_retrieval(input)
print(input)
print(response)

['Product Requirements Document\n\nProblem & Opportunity\n\nThe current user management system in Project Willy is limited to email/password login and lacks support for OAuth-based authentication methods like Google OAuth. Additionally, the existing system does not provide a comprehensive way for users to join different teams through invitations, nor does it allow team admins to manage team members effectively. The new enterprise-grade user management system for Omniflow users will address these gaps by introducing Google OAuth signup and login, team invitation mechanisms, and enhanced admin capabilities for managing team members and user account statuses.\n\nTarget Audience & User Persona\n\nThe target audience for the new user management system includes:\n\n\n\n\n\nCompany Admins: Responsible for managing user accounts, including suspending or deactivating accounts.\n\n\n\nTeam Admins: Responsible for managing team members, including adding and removing users from teams.\n\n\n\nRegul